#Trial 2

In [1]:
!pip install transformers datasets seqeval ray[tune]

     |████████████████████████████████| 2.0MB 12.1MB/s 
     |████████████████████████████████| 194kB 40.2MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 47.5MB 59kB/s 
     |████████████████████████████████| 890kB 29.0MB/s 
     |████████████████████████████████| 3.2MB 33.7MB/s 
     |████████████████████████████████| 245kB 40.5MB/s 
     |████████████████████████████████| 112kB 40.7MB/s 
     |████████████████████████████████| 1.3MB 31.3MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 133kB 42.2MB/s 
     |████████████████████████████████| 204kB 42.4MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 3.1MB 33.3MB/s 
     |████████████████████████████████| 317kB 39.0MB/s 
     |████████████████████████████████| 296kB 39.9MB/s 
     |████████████████████████████████| 143kB 40.9MB/s 
 

In [2]:
!wget http://noisy-text.github.io/2017/files/wnut17train.conll

--2021-03-26 09:32:13--  http://noisy-text.github.io/2017/files/wnut17train.conll
Resolving noisy-text.github.io (noisy-text.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to noisy-text.github.io (noisy-text.github.io)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 493781 (482K) [application/octet-stream]
Saving to: ‘wnut17train.conll’

wnut17train.conll   100%[===================>] 482.21K  --.-KB/s    in 0.02s   

2021-03-26 09:32:13 (25.0 MB/s) - ‘wnut17train.conll’ saved [493781/493781]



In [3]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

texts, tags = read_wnut('wnut17train.conll')

In [4]:
print(texts[0][10:17], tags[0][10:17], sep='\n')

['for', 'two', 'weeks', '.', 'Empire', 'State', 'Building']
['O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location']


In [5]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [6]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [7]:
tag2id

{'B-corporation': 9,
 'B-creative-work': 6,
 'B-group': 10,
 'B-location': 12,
 'B-person': 0,
 'B-product': 1,
 'I-corporation': 4,
 'I-creative-work': 2,
 'I-group': 5,
 'I-location': 11,
 'I-person': 3,
 'I-product': 7,
 'O': 8}

In [8]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [9]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100 
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [10]:
max(train_labels[0])
min(train_labels[0])

-100

In [11]:
import torch

class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

In [12]:
from seqeval.metrics import f1_score,accuracy_score
def my_compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    label_list=list(unique_tags)

        # Remove ignored index (special tokens)
    true_predictions = [
            [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
    true_labels = [
            [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

    #print(true_predictions,true_labels,sep="\n")

    print(f1_score(true_labels,true_predictions))

    #results = metric.compute(predictions=true_predictions, references=true_labels)

   # print(results.marco)
    return {'f1':f1_score(true_labels,true_predictions),
            'acc':accuracy_score(true_labels,true_predictions)}

In [13]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments,DistilBertForTokenClassification



training_args = TrainingArguments(
        output_dir=".",
        learning_rate=1e-5,  # config
        do_train=True,
        do_eval=True,
        evaluation_strategy="epoch", #no and steps
        load_best_model_at_end=True,
        num_train_epochs=4,  # config
        max_steps=-1,
        per_device_train_batch_size=16,  # config
        per_device_eval_batch_size=16,  # config
        warmup_steps=0,
        weight_decay=0.1,  # config
        logging_dir="./logs",
    )


model=DistilBertForTokenClassification.from_pretrained("distilbert-base-cased",num_labels=len(unique_tags))

def model_init():
    return DistilBertForTokenClassification.from_pretrained("distilbert-base-cased",num_labels=len(unique_tags))

trainer = Trainer(
    model_init=model_init,                        
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset   ,
    compute_metrics=my_compute_metrics
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [14]:
trainer.train()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss,F1,Acc,Runtime,Samples Per Second
1,No log,0.172095,0.097458,0.959192,4.206000,161.436000
2,No log,0.143974,0.381895,0.966032,4.183300,162.314000
3,0.244100,0.128867,0.440780,0.968908,4.203700,161.523000
4,0.244100,0.127598,0.460452,0.968986,4.183100,162.321000


0.09745762711864407
0.3818953323903819
0.4407796101949025
0.46045197740113


TrainOutput(global_step=680, training_loss=0.21114466330584358, metrics={'train_runtime': 221.141, 'train_samples_per_second': 3.075, 'total_flos': 382364210739600.0, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 633175, 'init_mem_gpu_alloc_delta': 261328384, 'init_mem_cpu_peaked_delta': 103274, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1195099, 'train_mem_gpu_alloc_delta': 1051874816, 'train_mem_cpu_peaked_delta': 89430959, 'train_mem_gpu_peaked_delta': 332573696})

In [15]:
trainer.evaluate()

0.46045197740113


{'epoch': 4.0,
 'eval_acc': 0.9689856198989506,
 'eval_f1': 0.46045197740113,
 'eval_loss': 0.12759794294834137,
 'eval_mem_cpu_alloc_delta': 3370992,
 'eval_mem_cpu_peaked_delta': 1156888,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 47949824,
 'eval_runtime': 4.2756,
 'eval_samples_per_second': 158.808}

In [16]:
from ray import tune
from ray.tune import track

In [17]:
tune_config = {
        "per_device_train_batch_size": tune.grid_search([16,32]),
        "per_device_eval_batch_size": tune.grid_search([16,32]),
        "num_train_epochs": tune.grid_search([2, 3, 4]),
        "learning_rate":tune.grid_search([1e-5,2e-5]),
        "max_steps": 1 
    }

In [18]:
from ray.tune.schedulers import ASHAScheduler
 
custom_scheduler = ASHAScheduler(
    metric='eval_f1',
    mode="max",
)

In [20]:
pip install GPy sklearn

  Created wheel for GPy: filename=GPy-1.9.9-cp37-cp37m-linux_x86_64.whl size=2626957 sha256=bdd27612e69ab890f678121b77daaa3ceb97c5da85724d06588539a83b195cd0
  Stored in directory: /root/.cache/pip/wheels/5d/36/66/2b58860c84c9f2b51615da66bfd6feeddbc4e04d887ff96dfa
  Created wheel for paramz: filename=paramz-0.9.5-cp37-none-any.whl size=102552 sha256=8bd78e0fac9418ff5d829ac38787db568ab5071f47b94fe426a427bcd8f03e65
  Stored in directory: /root/.cache/pip/wheels/c8/4a/0e/6e0dc85541825f991c431619e25b870d4b812c911214690cf8
Successfully built GPy paramz


In [24]:
from ray.tune.schedulers.pb2 import PB2

pb2_scheduler = PB2(
        time_attr='time_total_s',
        metric='eval_f1',
        mode='max',
        perturbation_interval=600.0,
        hyperparam_bounds={
            "learning_rate": [1e-5, 1e-3]   }
            )

In [25]:
trainer.hyperparameter_search(n_trials=1,direction="maximize", hp_space=lambda _: tune_config,  scheduler=pb2_scheduler)

You are using PopulationBasedTraining but you haven't enabled checkpointing. This means your trials will train from scratch everytime they are exploiting new configurations. Consider enabling checkpointing by passing `keep_checkpoints_num=1` as an additional argument to `Trainer.hyperparameter_search`.


== Status ==
Memory usage on this node: 5.1/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 1/2 CPUs, 1/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_inner_2021-03-26_09-41-13
Number of trials: 1/24 (1 RUNNING)
+--------------------+----------+-------+-----------------+--------------------+------------------------------+-------------------------------+
| Trial name         | status   | loc   |   learning_rate |   num_train_epochs |   per_device_eval_batch_size |   per_device_train_batch_size |
|--------------------+----------+-------+-----------------+--------------------+------------------------------+-------------------------------|
| _inner_672b1_00000 | RUNNING  |       |           1e-05 |                  2 |                           16 |                            16 |
+--------------------+----------+-------+-----------------+--------------------+------------------------------+----

2021-03-26 09:41:33,020	WARNING worker.py:1107 -- The actor or task with ID ffffffffffffffffcc0320e00aa0584cd481d0b901000000 cannot be scheduled right now. It requires {CPU: 1.000000}, {GPU: 1.000000} for placement, but this node only has remaining {1.000000/2.000000 CPU, 7.324219 GiB/7.324219 GiB memory, 0.000000/1.000000 GPU, 1.000000/1.000000 accelerator_type:K80, 1.000000/1.000000 node:172.28.0.2, 2.490234 GiB/2.490234 GiB object_store_memory}
. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.
(pid=575) 2021-03-26 09:41:35.125307: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=575) Some weights of the model checkpoint at distilbert-base-

(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) <IPython.core.display.HTML object>
(pid=575) 

(pid=575) Error in atexit._run_exitfuncs:
(pid=575) Traceback (most recent call last):
(pid=575)   File "/usr/local/lib/python3.7/dist-packages/IPython/core/history.py", line 786, in writeout_cache
(pid=575)     self._writeout_input_cache(conn)
(pid=575)   File "/usr/local/lib/python3.7/dist-packages/IPython/core/history.py", line 770, in _writeout_input_cache
(pid=575)     (self.session_number,)+line)
(pid=575) sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140497228334848 and this is thread id 140501624792960.
(pid=602) 2021-03-26 09:41:55.692320: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=602) Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias

...

(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.HTML object>
(pid=1181) <IPython.core.display.H

2021-03-26 09:49:45,358	INFO tune.py:450 -- Total run time: 512.24 seconds (512.22 seconds for the tuning loop).


== Status ==
Memory usage on this node: 5.5/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_inner_2021-03-26_09-41-13
Number of trials: 24/24 (24 TERMINATED)
+--------------------+------------+-------+-----------------+--------------------+------------------------------+-------------------------------+-------------+
| Trial name         | status     | loc   |   learning_rate |   num_train_epochs |   per_device_eval_batch_size |   per_device_train_batch_size |   objective |
|--------------------+------------+-------+-----------------+--------------------+------------------------------+-------------------------------+-------------|
| _inner_672b1_00000 | TERMINATED |       |           1e-05 |                  2 |                           16 |                            16 |    0.140056 |
| _inner_672b1_00001 | TERMINATED |       |   

2021-03-26 09:49:45,569	WARNING worker.py:1107 -- A worker died or was killed while executing task ffffffffffffffffdbda44882cfe1588004f2a5501000000.


BestRun(run_id='672b1_00015', objective=0.6418494052171354, hyperparameters={'per_device_train_batch_size': 32, 'per_device_eval_batch_size': 16, 'num_train_epochs': 3, 'learning_rate': 2e-05, 'max_steps': 1})

In [34]:
training_args1 = TrainingArguments(
        output_dir="res",
        learning_rate=2e-05,  # config
        num_train_epochs=3,  # config
        max_steps=1,
        per_device_train_batch_size=32,  # config
        per_device_eval_batch_size=16,  # config
        logging_dir="./logs1",
    )

In [35]:
trainer1 = Trainer(
    model_init=model_init,                        
    args=training_args1,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset   ,
    compute_metrics=my_compute_metrics
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [36]:
trainer1.train()

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Step,Training Loss


TrainOutput(global_step=1, training_loss=2.5268771648406982, metrics={'train_runtime': 0.5443, 'train_samples_per_second': 1.837, 'total_flos': 1126671707520.0, 'epoch': 0.01, 'init_mem_cpu_alloc_delta': 356544, 'init_mem_gpu_alloc_delta': 261968384, 'init_mem_cpu_peaked_delta': 187721, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 463617, 'train_mem_gpu_alloc_delta': 802158080, 'train_mem_cpu_peaked_delta': 246525, 'train_mem_gpu_peaked_delta': 373754368})

In [37]:
trainer1.evaluate()

0.0057097541633624104


{'epoch': 0.01,
 'eval_acc': 0.30656820831713955,
 'eval_f1': 0.0057097541633624104,
 'eval_loss': 2.323000907897949,
 'eval_mem_cpu_alloc_delta': 3384904,
 'eval_mem_cpu_peaked_delta': 2804260,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 47862272,
 'eval_runtime': 4.3203,
 'eval_samples_per_second': 157.165}